In [60]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [61]:
# perform GET request to the page
r = requests.get("https://www.tfrrs.org/all_performances/IN_college_m_Indiana_IN.html?list_hnd=4153&amp;season_hnd=608")

# use .content method to extract html content and parse it
soup = BeautifulSoup(r.content, "html.parser")

# return the first htmlElement with the myTable id, in this case the 100m table
hundred_table = soup.find(id="myTable")

# td element are the data entries in the table
entries = hundred_table.find_all('td')

print(len(entries))
print()
print(entries[1])

154

<td class="tablesaw-priority-1">
<a data-turbo="false" data-turbo-frame="_top" href="https://www.tfrrs.org/athletes/7989193/Indiana_IN/Antonio_Laidler">Laidler, Antonio</a>
</td>


In [62]:
items = []

for i in range(len(entries)):
    # If i is 0 or divisible by 7 we want to skip it. This is due to the 0th column being blank in the table
    if not i % 7: 
        continue
    # using the .text method to get the innerHTML value and the .strip() because some text has new line charecters attached to the data
    items.append(entries[i].text.strip()) 

print(items[0:5])

['Laidler, Antonio', 'JR-3', '10.19', 'NCAA East Preliminary Round', 'May 25, 2023']


In [63]:
# create empty dataframe with column names
df = pd.DataFrame(columns=["Athlete", "Year", "Time", "Meet", "Meet Date", "Wind"])

# each grouping of 6 items is a row in the dataframe
for i in range(5, len(items), 6):
    tempDf = pd.DataFrame([{"Athlete": items[i - 5], "Year": items[i - 4], "Time": items[i - 3],
                            "Meet": items[i - 2], "Meet Date": items[i - 1], "Wind": items[i]}])
    # update the current dataframe with the new row
    df = pd.concat([df, tempDf], ignore_index=True)

df.head()

,Athlete,Year,Time,Meet,Meet Date,Wind
0,"Laidler, Antonio",JR-3,10.19,NCAA East Preliminary Round,"May 25, 2023",1.9
1,"Laidler, Antonio",JR-3,10.20,Jim Freeman Louisville Invitational,"Apr 14, 2023",1.6
2,"Grant, Christopher",JR-3,10.28,Jim Freeman Louisville Invitational,"Apr 14, 2023",1.6
3,"Laidler, Antonio",JR-3,10.30,Big Ten Outdoor Championships,"May 12, 2023",+0.0
4,"Grant, Christopher",JR-3,10.33,2023 Billy Hayes Invitational,"May 5, 2023",0.5


In [64]:
# apply name modifications
df["Athlete"] = df["Athlete"].apply(lambda x: " ".join(x.replace(" ", "").split(",")[::-1]))

# apply year modifications
df["Year"] = df["Year"].apply(lambda x: x.split("-")[-1])

#apply meet date modifications, assuming "epoch date" is seconds since Unix epoch
df["Meet Date"] = pd.to_datetime(df["Meet Date"], format="%b %d, %Y").astype(np.int64) / int(1e9)

In [65]:
df

,Athlete,Year,Time,Meet,Meet Date,Wind
0,Antonio Laidler,3,10.19,NCAA East Preliminary Round,1.684973e+09,1.9
1,Antonio Laidler,3,10.20,Jim Freeman Louisville Invitational,1.681430e+09,1.6
2,Christopher Grant,3,10.28,Jim Freeman Louisville Invitational,1.681430e+09,1.6
3,Antonio Laidler,3,10.30,Big Ten Outdoor Championships,1.683850e+09,+0.0
4,Christopher Grant,3,10.33,2023 Billy Hayes Invitational,1.683245e+09,0.5
5,Christopher Grant,3,10.38,Pepsi Florida Relays,1.680221e+09,1.1
6,Antonio Laidler,3,10.38,Big Ten Outdoor Championships,1.683850e+09,-0.4
7,Christopher Grant,3,10.41,2023 Billy Hayes Invitational,1.683245e+09,1.9
8,Antonio Laidler,3,10.41,Pepsi Florida Relays,1.680221e+09,1.5
9,Christopher Grant,3,10.48,Big Ten Outdoor Championships,1.683850e+09,0.1


In [66]:
df.to_csv('trackscraper.csv', index=False)